In [1]:
using DPNeurifyFV, LazySets, PyVnnlib, NeuralVerification, JuMP, 
      LinearAlgebra, Flux, Gurobi
import VNNLib.NNLoader as NNL

const NV = NeuralVerification
const DP = DPNeurifyFV

# initialize all Gurobi models with the same env, 
# s.t. we don't print the license information all the time
const GRB_ENV = Gurobi.Env()

Academic license - for non-commercial use only - expires 2023-11-26


  ** incremental compilation may be fatally broken for this module **



precompiling...


node input_Sub: sub constant

parsing flatten :-)


parsing Matmul with input :-)


...

i: 1 - 

[-4.01316499710083, 2269.2729241943107], 7.289913177490234 sec


i: 2 - [-4.01316499710083, 1122.462314027753], 10.970319032669067 sec
i: 3 - [-4.01316499710083, 1095.465432296466], 10.976763010025024 sec
Steps: 3 - [-1030.3663799624544, 4.01316499710083] -> inconclusive


Academic license - for non-commercial use only - expires 2023-11-26
Academic license - for non-commercial use only - expires 2023-11-26


Gurobi.Env(Ptr{Nothing} @0x00000000040b4c70, false, 0)

In [2]:
vnnlib_path = "../vnncomp2023_benchmarks/benchmarks/acasxu/vnnlib/prop_1.vnnlib.gz"
onnx_path   = "../vnncomp2023_benchmarks/benchmarks/acasxu/onnx/ACASXU_run2a_1_1_batch_2000.onnx"

speclist = PyVnnlib.generate_specs(vnnlib_path, dtype=Float32)
specs = DP.generate_specs(speclist)
input_set, output_set = specs[1]

nn = NNL.load_network_dict(DP.CGType, onnx_path);

Warning! Output constraints have to be polytopes, found b with size (1, 1) - trying vec(b)


/home/philipp/.julia/conda/3/lib/python3.10/site-packages/vnnlib/parser.py:213: UserWarning: literal negation does not strictly follow SMT-LIB
  warnings.warn("literal negation does not strictly follow SMT-LIB")


node input_Sub: sub constant

parsing flatten :-)
parsing Matmul with input :-)


node Operation_1_Add: add constant

parsing ReLU :-)
parsing Matmul with input :-)
node Operation_2_Add: add constant

parsing ReLU :-)
parsing Matmul with input :-)
node Operation_3_Add: add constant

parsing ReLU :-)
parsing Matmul with input :-)
node Operation_4_Add: add constant

parsing ReLU :-)
parsing Matmul with input :-)
node Operation_5_Add: add constant

parsing ReLU :-)
parsing Matmul with input :-)
node Operation_6_Add: add constant

parsing ReLU :-)
parsing Matmul with input :-)
node linear_7_Add: add constant



["input"]["linear_7_Add"]

Constructing the whole NN -- yay -- :-)
inputs: ["input"]
outputs: ["linear_7_Add"]
input_shape = Any[1, 1, 1, 5], output_shape = Any[1, 5]


In [3]:
sz = DP.SplitZonotope(input_set, (5, 1, 1, 1))

DPNeurifyFV.SplitZonotope{Float64}(Zonotope{Float64, Vector{Float64}, Matrix{Float64}}([0.6399288845, 0.0, 0.0, 0.475, -0.475], [0.039928884499999984 0.0 … 0.0 0.0; 0.0 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.025000000000000022 0.0; 0.0 0.0 … 0.0 0.024999999999999967]), Dict{String, Vector}(), Dict{String, Tuple{Vararg{Vector{Float64}}}}("input" => ([0.6, -0.5, -0.5, 0.44999999999999996, -0.49999999999999994], [0.679857769, 0.5, 0.5, 0.5, -0.45])), [("input", 1), ("input", 2), ("input", 3), ("input", 4), ("input", 5)], Matrix{Float64}(undef, 0, 0), Float64[], (5, 1, 1, 1))

In [4]:
ẑ = DP.propagate(DP.LSTMSolver(), nn, sz)

z₁, z₂ = DP.split_split_zonotope(ẑ, nn.input_shape)

ẑ₁ = DP.propagate(DP.LSTMSolver(), nn, z₁)
ẑ₂ = DP.propagate(DP.LSTMSolver(), nn, z₂)

model = DP.build_model(ẑ₁)
optimize!(model)
ub₁ = objective_value(model)

model = DP.build_model(ẑ₂)
optimize!(model)
ub₂ = objective_value(model)

@show ub₁
@show ub₂

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(5 => 50)
│   summary(x) = 5×1 Matrix{Float64}
└ @ Flux /home/philipp/.julia/packages/Flux/n3cOc/src/layers/stateless.jl:60


Academic license - for non-commercial use only - expires 2023-11-26
Academic license - for non-commercial use only - expires 2023-11-26
ub₁ = 3381.7083218551397
ub₂ = 4899.564692676977


4899.564692676977

In [5]:
function bounds(sz::DP.SplitZonotope; skip_lp=false, upper=true, opt=() -> Gurobi.Optimizer(GRB_ENV))
    if skip_lp
        H = overapproximate(sz.z, Hyperrectangle)
        bds = upper ? high(H) : low(H)
    else
        # only for flat vectors for now, ignore batch dim
        n_out = sz.shape[1]
        bds = zeros(n_out)
        for i in 1:n_out
            model = DP.build_model(sz, opt_idx=i, maximize=upper, opt=opt)
            optimize!(model)
            bds[i] = objective_value(model)
        end
    end

    return bds
end

bounds (generic function with 1 method)

In [14]:
function contained_within_polytope_sz(nn::DP.CompGraph, input_set::AbstractHyperrectangle, polytope, params::DP.PriorityOptimizerParameters;
    solver=DPNFV(), split=split_largest_interval, concrete_sample=:Center)
    A, b = tosimplehrep(polytope)
    nn_spec = DP.merge_into_network(nn, A, b)

    in_shape = map(x -> ifelse(isa(x, Integer), x, 1), nn_spec.input_shape)
    s = DP.SplitZonotope(input_set, in_shape)

    function approximate_optimize_cell(cell)       
        out_cell = DP.propagate(solver, nn_spec, cell)
        violations = bounds(out_cell, upper=true)
        max_violation = maximum(violations)

        return max_violation, out_cell
    end

    achievable_value = cell -> begin
        l, u = cell.bounds["input"]
        center = 0.5 .* (l .+ u)
        y = maximum(DP.propagate(nn_spec, center))
        return center, y
    end

    return DP.general_priority_optimization(s, approximate_optimize_cell, achievable_value, params, true,
            split=split, bound_threshold_realizable=0., bound_threshold_approximate=0.)
end

contained_within_polytope_sz (generic function with 1 method)

In [7]:
# just a dummy method for now
function DP.split_multiple_times(sz::DP.SplitZonotope, n)
    return [sz]
end

In [8]:
function LazySets.radius(sz::DP.SplitZonotope)
    l, u = sz.bounds["input"]
    H = Hyperrectangle(low=l, high=u)
    return radius(H)
end

In [9]:
const CNT = Ref(0)

Base.RefValue{Int64}(0)

In [13]:
CNT[] = 0
params = DP.PriorityOptimizerParameters(max_steps=15000, print_frequency=100, stop_frequency=1, verbosity=2, timeout=1800)
solver = DP.LSTMSolver()


split_method = sz -> DP.split_split_zonotope(sz, nn.input_shape)


out_spec = HPolytope([1. 0 0 0 0;], [0.])
contained_within_polytope_sz(nn, input_set, out_spec, params, split=split_method, solver=solver)

i: 100 - [-0.020680464804172516, 3508.366693359719], 1.1357848644256592 sec


i: 200 - [-0.020680464804172516, 3372.785340275339], 1.9647369384765625 sec


i: 300 - [-0.020680464804172516, 3428.2264401146713], 3.036409854888916 sec


i: 400 - [-0.020680464804172516, 3283.677843188986], 4.202816963195801 sec


i: 500 - [-0.020680464804172516, 3248.8190015944097], 5.214087963104248 sec


...

i: 14700 - [-0.020229337736964226, 2757.8413647796233], 178.03419995307922 sec


i: 14800 - [-0.020229337736964226, 2764.1105288203144], 179.1244080066681 sec


i: 14900 - [-0.020229337736964226, 2782.3177422123863], 180.26474785804749 sec


i: 15000 - [-0.020229337736964226, 2754.841288270223], 181.35587692260742 sec


([0.6399288845, 0.0, -0.25, 0.475, -0.475], -0.020229338f0, 2807.1600227768076, 15000)

In [15]:
LSTM(3 => 5)

Recur(
  LSTMCell(3 => 5),                     # 190 parameters
)         # Total: 5 trainable arrays, 190 parameters,
          # plus 2 non-trainable, 10 parameters, summarysize 1.062 KiB.